In [3]:
import json

# Load unique questions from data.jsonl
with open('data.jsonl', 'r') as f:
    data_questions = set(json.loads(line)['Question'] for line in f)

# Filter and deduplicate judge1.jsonl based on "Question"
seen_questions = set()
filtered_entries = []

with open('judge1.jsonl', 'r') as f:
    for line in f:
        entry = json.loads(line)
        question = entry.get('Question')
        if question in data_questions and question not in seen_questions:
            seen_questions.add(question)
            filtered_entries.append(entry)

# Overwrite judge1.jsonl with filtered, deduplicated entries
with open('judge1.jsonl', 'w') as f:
    for entry in filtered_entries:
        f.write(json.dumps(entry) + '\n')

print(f"Filtered and deduplicated judge1.jsonl to {len(filtered_entries)} entries.")


Filtered and deduplicated judge1.jsonl to 1945 entries.


In [ ]:
!pip install transformers datasets scikit-learn pandas numpy sentence-transformers pyyaml openpyxl accelerate

In [2]:
from transformers import  AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
import pandas as pd
import numpy as np
import warnings
import logging
import random
import torch
import json
import yaml
import os
import json
import torch
from transformers import pipeline
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
warnings.filterwarnings("ignore")

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [4]:
import pandas as pd
contexts_file="rag.xlsx"
# Load contexts into a dictionary from Excel file
context_df = pd.read_excel(contexts_file)

# Ensure the necessary columns are present
if 'Question' not in context_df.columns or 'retrieved_context' not in context_df.columns:
    raise ValueError("Excel file must contain 'Question' and 'retrieved_context' columns.")

# Build the dictionary
context_dict = dict(zip(context_df['Question'], context_df['retrieved_context']))


In [ ]:
# import json

# with open('result_qwen7b_main_architecture05032025.jsonl', 'r') as f:
#     for i, line in enumerate(f, 1):
#         data = json.loads(line)
#         print(f"Line {i}: {len(data.keys())} keys")


In [5]:
contexts_file1="result_qwen7b_main_architecture05032025.jsonl"
context_dict1 = {}
with open(contexts_file1, 'r', encoding='utf-8') as cf:
    for line in cf:
        entry = json.loads(line)
        question = entry.get('Question')
        retrieved_context = entry.get('main_model_reasoning')
        if question and retrieved_context:
            context_dict1[question] = retrieved_context

In [6]:
data_file = config['actual_data']
data_list = []
with open(data_file, 'r', encoding='utf-8') as df:
    for line in df:
        data_list.append(json.loads(line))

In [7]:
def get_data_point_with_context_list(data_entry):

    record = data_entry
    question = record.get('Question')
    retrieved_context = context_dict.get(question)
    critique_content=context_dict1.get(question)
    record_with_context = record.copy()
    record_with_context['main_model_reasoning']=critique_content
    record_with_context['retrieved_context'] = retrieved_context
    
    return record_with_context

In [8]:
point_line=get_data_point_with_context_list(data_list[0])

In [ ]:
point_line

In [9]:
Critique_system_prompt = '''
You are a legal critique expert. A question has been answered based on the document provided in the retrieved_context.

Your task: Critically assess the reasoning and answer in light of the retrieved_context. Identify any logical gaps, potential misinterpretations of the law, or areas where the reasoning could be challenged or improved.

Additionally, evaluate each of the provided answer options individually. Where appropriate, suggest a better answer with well-reasoned arguments grounded in law.

Base 90% of your critique strictly on the retrieved_context, and use up to 10% general legal knowledge to support your evaluation.
'''


In [10]:
def generate_critique_content(data_entry):
    final_content = f"""Question:
{data_entry['Question']}

Options:
A: {data_entry['A']}
B: {data_entry['B']}
C: {data_entry['C']}
D: {data_entry['D']}

Retrieved Context (Laws/Regulations):
{data_entry['retrieved_context']}

Initial Answer:
{data_entry['Correct Answer']}

Initial Reasoning:
{data_entry['main_model_reasoning']}

👉 Your task: Critically evaluate the reasoning and answer above. Identify any logical gaps, potential misinterpretations of the law, or areas where the reasoning may be challenged or require clarification. 
Focus your critique 90% on the retrieved context and 10% on general legal knowledge.
"""
    return final_content


In [ ]:
from huggingface_hub import login
login('')

In [12]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [13]:
def critique_model(data_entry):
    content = generate_critique_content(data_entry)
    messages = [
        {"role": "system", "content": Critique_system_prompt},
        {"role": "user", "content": content}
    ]
    outputs = pipe(
        messages,
        max_new_tokens=1500,
    )
    critique=outputs[0]["generated_text"][-1]['content']
    
    return critique

In [ ]:
import os
import json

result_log_path = 'Critique.jsonl'  # Update with your file path

# Load existing results if available
if os.path.isfile(result_log_path):
    with open(result_log_path, 'r', encoding='utf-8') as file:
        output_log = [json.loads(line) for line in file]
else:
    output_log = []

# Create a set of already processed questions to avoid duplicates
processed_questions = set(entry["Question"] for entry in output_log)
i = 0

for datapoint in data_list:
    question = datapoint.get('Question')
    if question in processed_questions:
        print(f"Skipping already processed question: {question}")
        continue

    try:
        compact_data = get_data_point_with_context_list(datapoint)
        critique = critique_model(compact_data)

        temp_data = {
            "Question": question,
            "Critique": critique
        }

        # Add to log list and checkpoint to JSONL file
        output_log.append(temp_data)
        with open(result_log_path, 'a', encoding='utf-8') as file:
            file.write(json.dumps(temp_data, ensure_ascii=False) + '\n')

        print(f"Data {i} saved")
        i += 1

    except Exception as e:
        print(f"Error processing question: {question}")
        print(f"Exception: {e}")

print("Processing complete.")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping already processed question: What legal actions should the authorities consider based on the specific allegations of assault highlighted in the passage, particularly concerning the victim's age and the nature of the crimes?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Data 0 saved


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Data 733 saved


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Data 734 saved


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Data 735 saved


In [14]:
import os
import json

result_log_path = 'Critique.jsonl'  # Update with your file path

# Load existing results if available
if os.path.isfile(result_log_path):
    with open(result_log_path, 'r', encoding='utf-8') as file:
        output_log = [json.loads(line) for line in file]
else:
    output_log = []

# Create a set of already processed questions to avoid duplicates
processed_questions = set(entry["Question"] for entry in output_log)
i=0
for datapoint in data_list:
    if datapoint['Question'] in processed_questions:
        print(f"Skipping already processed question: {datapoint['Question']}")
        continue

    compact_data = get_data_point_with_context_list(datapoint)
    critique = critique_model(compact_data)
    
    temp_data = {"Question": datapoint["Question"]}
    temp_data['Critique'] = critique

    # Add to log list and checkpoint to JSONL file
    output_log.append(temp_data)
    with open(result_log_path, 'a', encoding='utf-8') as file:
        file.write(json.dumps(temp_data, ensure_ascii=False) + '\n')

    print(f"Data {i} saved")
    i+=1

print("Processing complete.")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Data 0 saved


KeyboardInterrupt: 